# 3. Imporve NN

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt

%matplotlib inline

C:\ProgramData\Anaconda3\envs\nlp\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 3.1 Load data

In [35]:
data_root = r'C:\Users\7153678\Desktop\AI\src\nlp\data\mnist'
pickle_file = os.path.join(data_root, 'notMNIST.pickle')

image_size = 28
class_num = 10

In [36]:
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_data = save['train_data']
    train_labels = save['train_labels']
    valid_data = save['valid_data']
    valid_labels = save['valid_labels']
    test_data = save['test_data']
    test_labels = save['test_labels']
    del save
    
print('train: {} and {}'.format(train_data.shape, train_labels.shape))
print('valid: {} and {}'.format(valid_data.shape, valid_labels.shape))    
print('test: {} and {}'.format(test_data.shape, test_labels.shape)) 

train: (200000, 28, 28) and (200000,)
valid: (10000, 28, 28) and (10000,)
test: (10000, 28, 28) and (10000,)


In [37]:
def reformat(data, labels):
    new_data = data.reshape(-1, image_size**2)
    new_label = (labels[:, None] == np.arange(class_num)).astype(np.float32)
    return new_data, new_label

train_data, train_labels = reformat(train_data, train_labels)
valid_data, valid_labels = reformat(valid_data, valid_labels)
test_data, test_labels = reformat(test_data, test_labels)
print('train: {} and {}'.format(train_data.shape, train_labels.shape))
print('valid: {} and {}'.format(valid_data.shape, valid_labels.shape))    
print('test: {} and {}'.format(test_data.shape, test_labels.shape))   

train: (200000, 784) and (200000, 10)
valid: (10000, 784) and (10000, 10)
test: (10000, 784) and (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return 100.0*sum(np.argmax(predictions, 1)==np.argmax(labels, 1))/labels.shape[0]

## 3.2 Regularization

In [16]:
batch_size = 128
graph = tf.Graph()
h1 = 1024
beta = 0.01
num_steps = 20001
learn_rate = 0.5

In [17]:
graph = tf.Graph()
with graph.as_default():
    # input, only train is with SGD
    tf_train_data = tf.placeholder(tf.float32, shape=(batch_size, image_size**2))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, class_num))
    tf_valid_data = tf.constant(valid_data)
    tf_test_data = tf.constant(test_data)
    
    # variables
    weights1 = tf.Variable(tf.truncated_normal([image_size**2, h1]))
    biases1 = tf.Variable(tf.zeros(h1))  
    weights2 = tf.Variable(tf.truncated_normal([h1, class_num]))
    biases2 = tf.Variable(tf.zeros(class_num))  
    
    # train compution   
    logits_1 = tf.matmul(tf_train_data, weights1) + biases1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits_2))
    regularization = beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    loss = tf.reduce_mean(loss + regularization)

    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)
    
    # prediction
    train_prediction = tf.nn.softmax(logits_2)
    
    logits_1 = tf.matmul(tf_valid_data, weights1) + biases1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights2) + biases2
    valid_prediction = tf.nn.softmax(logits_2)

    logits_1 = tf.matmul(tf_test_data, weights1) + biases1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights2) + biases2
    test_prediction = tf.nn.softmax(logits_2)

In [18]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initilazed')
    
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels.shape[0]-batch_size)
        # generate a mini-batch
        batch_data = train_data[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        
        # dict to feed mini-batch
        feed_dict = {tf_train_data: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        # logging
        if step % 500 == 0:
            print('Mini-batch train at step {} with loss: {}'.format(step, l))
            print('Mini-batch train accuracy: {}, valid accuracy: {}'.format(
                 accuracy(predictions, batch_labels),
                 accuracy(valid_prediction.eval(), valid_labels)))
    print('Mini-batch test accuarcy: {}'.format(accuracy(test_prediction.eval(), test_labels)))

Initilazed
Mini-batch train at step 0 with loss: 3490.550048828125
Mini-batch train accuracy: 9.375, valid accuracy: 27.53
Mini-batch train at step 500 with loss: 21.42230987548828
Mini-batch train accuracy: 75.78125, valid accuracy: 84.06
Mini-batch train at step 1000 with loss: 0.923163652420044
Mini-batch train accuracy: 84.375, valid accuracy: 83.82
Mini-batch train at step 1500 with loss: 0.8249024152755737
Mini-batch train accuracy: 82.8125, valid accuracy: 83.92
Mini-batch train at step 2000 with loss: 0.5819976329803467
Mini-batch train accuracy: 87.5, valid accuracy: 82.47
Mini-batch train at step 2500 with loss: 0.7443509101867676
Mini-batch train accuracy: 82.03125, valid accuracy: 82.41
Mini-batch train at step 3000 with loss: 0.8527445793151855
Mini-batch train accuracy: 78.90625, valid accuracy: 83.75
Mini-batch train at step 3500 with loss: 0.7074048519134521
Mini-batch train accuracy: 82.8125, valid accuracy: 83.69
Mini-batch train at step 4000 with loss: 0.712395191192

## 3.3 Overfitting

In [11]:
batch_size = 128
graph = tf.Graph()
h1 = 1024
beta = 0.01
num_steps = 20001
learn_rate = 0.5

train_data2 = train_data[:1000, :]
train_labels2 = train_labels[:1000, :]

In [12]:
g = tf.Graph()
with g.as_default():
    # input, only train is with SGD
    tf_train_data = tf.placeholder(tf.float32, shape=(batch_size, image_size**2))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, class_num))
    tf_valid_data = tf.constant(valid_data)
    tf_test_data = tf.constant(test_data)
    
    # variables
    weights1 = tf.Variable(tf.truncated_normal([image_size**2, h1]))
    biases1 = tf.Variable(tf.zeros(h1))  
    weights2 = tf.Variable(tf.truncated_normal([h1, class_num]))
    biases2 = tf.Variable(tf.zeros(class_num))  
    
    # train compution   
    logits_1 = tf.matmul(tf_train_data, weights1) + biases1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits_2))
    regularization = beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    loss = tf.reduce_mean(loss + regularization)

    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)
    
    # prediction
    train_prediction = tf.nn.softmax(logits_2)
    
    logits_1 = tf.matmul(tf_valid_data, weights1) + biases1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights2) + biases2
    valid_prediction = tf.nn.softmax(logits_2)

    logits_1 = tf.matmul(tf_test_data, weights1) + biases1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights2) + biases2
    test_prediction = tf.nn.softmax(logits_2)
    
with tf.Session(graph=g) as session:
    tf.global_variables_initializer().run()
    print('Initilazed')
    
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels2.shape[0]-batch_size)
        # generate a mini-batch
        batch_data = train_data2[offset:(offset+batch_size), :]
        batch_labels = train_labels2[offset:(offset+batch_size), :]
        
        # dict to feed mini-batch
        feed_dict = {tf_train_data: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        # logging
        if step % 500 == 0:
            print('Mini-batch train at step {} with loss: {}'.format(step, l))
            print('Mini-batch train accuracy: {}, valid accuracy: {}'.format(
                 accuracy(predictions, batch_labels),
                 accuracy(valid_prediction.eval(), valid_labels)))
    print('Mini-batch test accuarcy: {}'.format(accuracy(test_prediction.eval(), test_labels)))

Initilazed
Mini-batch train at step 0 with loss: 3439.205078125
Mini-batch train accuracy: 8.59375, valid accuracy: 28.96
Mini-batch train at step 500 with loss: 21.05567741394043
Mini-batch train accuracy: 100.0, valid accuracy: 80.36
Mini-batch train at step 1000 with loss: 0.5125932693481445
Mini-batch train accuracy: 100.0, valid accuracy: 80.82
Mini-batch train at step 1500 with loss: 0.35965293645858765
Mini-batch train accuracy: 100.0, valid accuracy: 81.19
Mini-batch train at step 2000 with loss: 0.3735182583332062
Mini-batch train accuracy: 100.0, valid accuracy: 81.15
Mini-batch train at step 2500 with loss: 0.43856197595596313
Mini-batch train accuracy: 97.65625, valid accuracy: 80.09
Mini-batch train at step 3000 with loss: 0.35190993547439575
Mini-batch train accuracy: 100.0, valid accuracy: 80.86
Mini-batch train at step 3500 with loss: 0.3493117094039917
Mini-batch train accuracy: 100.0, valid accuracy: 80.83
Mini-batch train at step 4000 with loss: 0.3305906653404236
Mi

## 3.4 Drop out

In [17]:
batch_size = 128
graph = tf.Graph()
h1 = 1024
beta = 0.01
num_steps = 30001
learn_rate = 0.5
keep_prob = 0.6

In [18]:
g = tf.Graph()
with g.as_default():
    # input, only train is with SGD
    tf_train_data = tf.placeholder(tf.float32, shape=(batch_size, image_size**2))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, class_num))
    tf_valid_data = tf.constant(valid_data)
    tf_test_data = tf.constant(test_data)
    
    # variables
    weights1 = tf.Variable(tf.truncated_normal([image_size**2, h1]))
    biases1 = tf.Variable(tf.zeros(h1))  
    weights2 = tf.Variable(tf.truncated_normal([h1, class_num]))
    biases2 = tf.Variable(tf.zeros(class_num))  
    
    # train compution   
    logits_1 = tf.matmul(tf_train_data, weights1) + biases1
    relu_layer = tf.nn.relu(logits_1)
    dropout_layer = tf.nn.dropout(relu_layer, keep_prob=keep_prob)
    logits_2 = tf.matmul(dropout_layer, weights2) + biases2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits_2))
    regularization = beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    loss = tf.reduce_mean(loss + regularization)

    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)
    
    # prediction
    train_prediction = tf.nn.softmax(logits_2)
    
    logits_1 = tf.matmul(tf_valid_data, weights1) + biases1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights2) + biases2
    valid_prediction = tf.nn.softmax(logits_2)

    logits_1 = tf.matmul(tf_test_data, weights1) + biases1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights2) + biases2
    test_prediction = tf.nn.softmax(logits_2)
    
with tf.Session(graph=g) as session:
    tf.global_variables_initializer().run()
    print('Initilazed')
    
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels.shape[0]-batch_size)
        # generate a mini-batch
        batch_data = train_data[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        
        # dict to feed mini-batch
        feed_dict = {tf_train_data: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        # logging
        if step % 1000 == 0:
            print('Mini-batch train at step {} with loss: {}'.format(step, l))
            print('Mini-batch train accuracy: {}, valid accuracy: {}'.format(
                 accuracy(predictions, batch_labels),
                 accuracy(valid_prediction.eval(), valid_labels)))
    print('Mini-batch test accuarcy: {}'.format(accuracy(test_prediction.eval(), test_labels)))

Initilazed
Mini-batch train at step 0 with loss: 3604.9814453125
Mini-batch train accuracy: 9.375, valid accuracy: 34.2
Mini-batch train at step 1000 with loss: 0.9797767400741577
Mini-batch train accuracy: 82.8125, valid accuracy: 83.46
Mini-batch train at step 2000 with loss: 0.6209619045257568
Mini-batch train accuracy: 86.71875, valid accuracy: 81.92
Mini-batch train at step 3000 with loss: 0.899412989616394
Mini-batch train accuracy: 80.46875, valid accuracy: 82.94
Mini-batch train at step 4000 with loss: 0.7836065888404846
Mini-batch train accuracy: 81.25, valid accuracy: 82.08
Mini-batch train at step 5000 with loss: 0.9031695127487183
Mini-batch train accuracy: 84.375, valid accuracy: 82.9
Mini-batch train at step 6000 with loss: 0.7172757983207703
Mini-batch train accuracy: 83.59375, valid accuracy: 83.49
Mini-batch train at step 7000 with loss: 0.8949711918830872
Mini-batch train accuracy: 78.125, valid accuracy: 82.85
Mini-batch train at step 8000 with loss: 0.74877750873565

In [19]:
train_data2 = train_data[:1000, :]
train_labels2 = train_labels[:1000, :]
g = tf.Graph()
with g.as_default():
    # input, only train is with SGD
    tf_train_data = tf.placeholder(tf.float32, shape=(batch_size, image_size**2))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, class_num))
    tf_valid_data = tf.constant(valid_data)
    tf_test_data = tf.constant(test_data)
    
    # variables
    weights1 = tf.Variable(tf.truncated_normal([image_size**2, h1]))
    biases1 = tf.Variable(tf.zeros(h1))  
    weights2 = tf.Variable(tf.truncated_normal([h1, class_num]))
    biases2 = tf.Variable(tf.zeros(class_num))  
    
    # train compution   
    logits_1 = tf.matmul(tf_train_data, weights1) + biases1
    relu_layer = tf.nn.relu(logits_1)
    dropout_layer = tf.nn.dropout(relu_layer, keep_prob=keep_prob)
    logits_2 = tf.matmul(dropout_layer, weights2) + biases2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits_2))
    regularization = beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    loss = tf.reduce_mean(loss + regularization)

    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)
    
    # prediction
    train_prediction = tf.nn.softmax(logits_2)
    
    logits_1 = tf.matmul(tf_valid_data, weights1) + biases1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights2) + biases2
    valid_prediction = tf.nn.softmax(logits_2)

    logits_1 = tf.matmul(tf_test_data, weights1) + biases1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights2) + biases2
    test_prediction = tf.nn.softmax(logits_2)
    
with tf.Session(graph=g) as session:
    tf.global_variables_initializer().run()
    print('Initilazed')
    
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels2.shape[0]-batch_size)
        # generate a mini-batch
        batch_data = train_data2[offset:(offset+batch_size), :]
        batch_labels = train_labels2[offset:(offset+batch_size), :]
        
        # dict to feed mini-batch
        feed_dict = {tf_train_data: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        # logging
        if step % 1000 == 0:
            print('Mini-batch train at step {} with loss: {}'.format(step, l))
            print('Mini-batch train accuracy: {}, valid accuracy: {}'.format(
                 accuracy(predictions, batch_labels),
                 accuracy(valid_prediction.eval(), valid_labels)))
    print('Mini-batch test accuarcy: {}'.format(accuracy(test_prediction.eval(), test_labels)))

Initilazed
Mini-batch train at step 0 with loss: 3607.59912109375
Mini-batch train accuracy: 7.8125, valid accuracy: 37.18
Mini-batch train at step 1000 with loss: 0.5174341201782227
Mini-batch train accuracy: 100.0, valid accuracy: 81.01
Mini-batch train at step 2000 with loss: 0.39363279938697815
Mini-batch train accuracy: 100.0, valid accuracy: 81.11
Mini-batch train at step 3000 with loss: 0.3683589994907379
Mini-batch train accuracy: 100.0, valid accuracy: 80.91
Mini-batch train at step 4000 with loss: 0.34328383207321167
Mini-batch train accuracy: 100.0, valid accuracy: 81.25
Mini-batch train at step 5000 with loss: 0.4035557508468628
Mini-batch train accuracy: 99.21875, valid accuracy: 80.73
Mini-batch train at step 6000 with loss: 0.35773080587387085
Mini-batch train accuracy: 100.0, valid accuracy: 80.94
Mini-batch train at step 7000 with loss: 0.36978304386138916
Mini-batch train accuracy: 100.0, valid accuracy: 81.14
Mini-batch train at step 8000 with loss: 0.360526829957962

## 3.5 Imporve with deeper NN 

### 3.5.1 Try high level API: Estimators

In [5]:
tf.logging.set_verbosity(tf.logging.INFO)
from __future__ import absolute_import, division, print_function, unicode_literals

In [56]:
def cnn_model_fn(features, labels, mode):
    '''Model function of CNN'''
    
    # Input layer
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1])
    print(input_layer.shape)
    
    # Convolutional layer #1
    conv1 = tf.layers.conv2d(
        inputs = input_layer, 
        filters=32,
        kernel_size=[5, 5], 
        padding='same', 
        activation=tf.nn.relu)
    
    # Pooling layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Convolutional layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1, 
        filters=64, 
        kernel_size=[5, 5], 
        padding='same', 
        activation=tf.nn.relu)
      
    # Pooling layer #1
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    # Dense layer #1
    pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, 
                                training=mode == tf.estimator.ModeKeys.TRAIN)
    
    #Logits layer
    logits = tf.layers.dense(inputs=dropout, units=class_num)
    
    predictions = {
        'classes': tf.argmax(input=logits, axis=1),
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logits))
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
        
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=tf.argmax(input=labels, axis=1),
                                        predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode,loss=loss,
                                      eval_metric_ops=eval_metric_ops)


In [57]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir=data_root)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025C81518E48>, '_evaluation_master': '', '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None, '_task_id': 0, '_save_summary_steps': 100, '_train_distribute': None, '_device_fn': None, '_task_type': 'worker', '_global_id_in_cluster': 0, '_session_config': None, '_model_dir': 'C:\\Users\\7153678\\Desktop\\AI\\src\\nlp\\data\\mnist', '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_master': ''}


In [58]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [46]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

In [59]:
# train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
(100, 28, 28, 1)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\7153678\Desktop\AI\src\nlp\data\mnist\model.ckpt-1001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into C:\Users\7153678\Desktop\AI\src\nlp\data\mnist\model.ckpt.
INFO:tensorflow:probabilities = [[0.15792854 0.08667378 0.09345014 0.0884434  0.08824697 0.09821492
  0.08851908 0.09016933 0.09645264 0.11190128]
 [0.07136316 0.08533433 0.15975645 0.12650672 0.0976068  0.13163954
  0.10275698 0.09097126 0.06976172 0.06430311]
 [0.08096241 0.0797606  0.12615493 0.07251994 0.12678356 0.11385694
  0.11093532 0.07703714 0.1448418  0.0671474 ]
 [0.11842261 0.1321772  0.09319384 0.11175149 0.09167321 0.10226443
  0.07473855 0.09487867 0.06182375 0.11907618]
 [0.07481819 0.16958211 0.09796752 0.1017

INFO:tensorflow:loss = 2.0368164, step = 1002
INFO:tensorflow:Saving checkpoints for 1002 into C:\Users\7153678\Desktop\AI\src\nlp\data\mnist\model.ckpt.
INFO:tensorflow:Loss for final step: 2.0368164.


In [60]:
mnist_classifier.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.
(100, 28, 28, 1)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\7153678\Desktop\AI\src\nlp\data\mnist\model.ckpt-1002
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1002 into C:\Users\7153678\Desktop\AI\src\nlp\data\mnist\model.ckpt.
INFO:tensorflow:loss = 2.0138888, step = 1003
INFO:tensorflow:global_step/sec: 2.03799
INFO:tensorflow:loss = 0.7248892, step = 1103 (49.069 sec)
INFO:tensorflow:global_step/sec: 2.12888
INFO:tensorflow:loss = 0.68806016, step = 1203 (46.973 sec)
INFO:tensorflow:global_step/sec: 2.02098
INFO:tensorflow:loss = 0.64603543, step = 1303 (49.488 sec)
INFO:tensorflow:global_step/sec: 2.29268
INFO:tensorflow:loss = 0.465855, step = 1403 (43.610 sec)
INFO:tensorflow:global_step/sec: 2.28331
INFO:tensorflow:loss = 0.46244156, step = 150

In [61]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    y=test_labels,
    num_epochs=1,
    shuffle=False)

test_results = mnist_classifier.evaluate(input_fn=test_input_fn)
print(test_results)

INFO:tensorflow:Calling model_fn.
(?, 28, 28, 1)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-09-06-14:59:57
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\7153678\Desktop\AI\src\nlp\data\mnist\model.ckpt-2002
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-09-06-15:00:11
INFO:tensorflow:Saving dict for global step 2002: accuracy = 0.9343, global_step = 2002, loss = 0.22989973
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2002: C:\Users\7153678\Desktop\AI\src\nlp\data\mnist\model.ckpt-2002
{'global_step': 2002, 'accuracy': 0.9343, 'loss': 0.22989973}
